# パッケージインストール

In [ ]:
!pip install onnxruntime

# YOLOXクローン

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX

In [ ]:
%cd YOLOX

!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .

# モデルダウンロード

In [ ]:
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_nano.pth

# コンフィグコピー

In [ ]:
!git clone https://github.com/Kazuhito00/YOLOX-ONNX-TFLite-Sample.git

In [6]:
!cp YOLOX-ONNX-TFLite-Sample/config/nano.py ./

# ONNX出力(Export ONNX Model)

In [ ]:
!python tools/export_onnx.py \
    -n yolox-nano \
    -c yolox_nano.pth \
    --output-name yolox_nano.onnx

# ONNX -> TensorFlow 変換

In [ ]:
!pip install onnx-tf

In [ ]:
!onnx-tf convert \
    -i yolox_nano.onnx \
    -o yolox_nano_pb

# TensorFlow -> TensorFlow-Lite 変換

In [ ]:
!pip install tf-nightly

In [11]:
import tensorflow as tf

In [ ]:
# ダイナミックレンジ量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open('yolox_nano_dynamic_range_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 半精度浮動小数点数の量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quantized_model = converter.convert()

open('yolox_nano_float16_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 完全整数量子化
import glob
import numpy as np
from PIL import Image

image_pathlist = glob.glob('/content/YOLOX-ONNX-TFLite-Sample/representative_dataset/*.jpg')
image_pathlist = image_pathlist[:100]

def representative_dataset():
    for test_image_path in image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quantized_model = converter.convert()

open('yolox_nano_int8_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:

# 完全整数量子化(入力含む)
import glob
import numpy as np
from PIL import Image

image_pathlist = glob.glob('/content/YOLOX-ONNX-TFLite-Sample/representative_dataset/*.jpg')
image_pathlist = image_pathlist[:100]

def representative_dataset():
    for test_image_path in image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quantized_model = converter.convert()

open('yolox_nano_only_int8_quantize.tflite', 'wb').write(tflite_quantized_model)